In [1]:
import copy
import random

# Exploring Nested Features in KerML

This is a notebook where I am fooling around with possibilities for using the sequence semantics of SysML v2 to explore different kinds of relationships between things in a universe. The "user model" of SysML v2 (M1, where libraries live) is the source of constraints and rules for combining the things, denoted by individual characters, into sequences.

One of the reasons I used characters as symbols for these individual things is it also helps to look at the sequences and individual items themselves, which is what happens with strings (sequences of characters treated as an object).

If Greek characters, capital letters, and numbers are added, this allows for a universe of atoms with about 80 members. This probably plenty to work with giving how quickly combinatorial spaces expand.

# Section 1 - Exploring the size of spaces

This is a section to just play around with the combinatorics of arranging the items of a universe into sequences to see how quickly things get out of hand.

### Section 1.1 - Universe Setup

Define the universe of things. The key to keeping models under control in terms of size is to try and keep this universe small. Population of the universe is thus probably better as a bottoms-up effort than a top-down one.

In [2]:
number_things = 80

Maxium depth of feature to consider.

In [3]:
feature_depth = 5

To make it easier to deal with some of the conceptual issues of the labeling of sequences, it may be helpful to assign numbers to sequences as indexed.

### Section 1.2 - Completely default types

The most basic calculations are for unconstrained results.

In [4]:
anything_possibilities = 0
for i in range(1, feature_depth + 1):
    anything_possibilities = anything_possibilities + number_things ** i
    
print("{:,d}".format(anything_possibilities))

3,318,278,480


## Section 1.3 - Exploring Library Elements

After the initial play with the sizes of sequences, this section tries to look at how the constraints on sequences brings the size of the space down.

#### Section 1.3.1 - Classifiers and Minimal Interpretations

The minimum interpretations of classifiers are of length 1 (any sequence in a Classifier has its 1-tail inside also, which means only the 1-tails are not tails of other sequences).

In [5]:
number_things_in_class = 20
assert number_things_in_class < number_things

classifier_possibilities = number_things_in_class

for i in range(2, feature_depth + 1):
    classifier_possibilities = classifier_possibilities + number_things ** i
    
print("{:,d}".format(classifier_possibilities))

3,318,278,420


#### Section 1.3.2 - Basics of Nesting Features

One key consideration is that "Anything" has no minimum interpretation so it cannot be used in defining either side of a FeaturePair. "Things" can be used however. Since things is all possible sequences in of things in the universe, its minimal interpretation is a sequence of length 2.

In [6]:
things_size = number_things ** 2
print("{:,d}".format(things_size))

6,400


##### Section 1.3.2.1 - Defaults for nesting features in pure KerML

The rules above lead to a "default" nested feature (one feature with another as its featuringType, and neither with a specific Type applied) of sequences of lengths no less than 4, all concatenations of sequences from the minimal interpretation of "things."

The base rules for feature sequences then allow for the insertion of any atom from the universe between numbers of the minimal 2-sequences from the minimal interpretation of things.

In [7]:
number_of_5_length_sequences = things_size * number_things
print("{:,d}".format(number_of_5_length_sequences))

512,000


##### Section 1.3.2.2 - Defaults for nesting features in SysML

In SysML, all part properties must be typed by Part Definition or a specialization. Since Part Definitions are Classifiers, their minimal interpretations are of length one. So the nesting will result in sequences of length no less than 2, simply combinations of all 1-sequences of Part Definition.

## Section 2 - Sets from the Library

Start capturing library sets and specialize appropriately.

### Section 2.1 - Occurrences

Occurrences specialize Classifier, and obey the same rules. Minimal interpretations are of length 1.

### Section 2.2 - Links

Links specialize objects which specialize occurrences. The links that are often the most useful are the binary links as they do a lot of heavy lifting in libraries to link entities.
Generic links have attributes that are two or more participants, which can be pulled from the Anything set of sequences.

#### Section 2.2.1 - Combinatorial issues

Links show the combinatorial explosion problem - if there are many sequences under Links and the participants field can hold any number of references from one link sequence to another, then numbers grow incredibly fast.

#### Section 2.2.2 - BinaryLinks

The binary links are simpler to deal with in that there are exactly two participants. Each binary link effectively pairs sequences.

#### Section 2.2.2.1 - Nesting and ends

The sequences within the interpretations of different features of the Association are useful for puzzling out end semantics.

Here we generate the list of possibilities. Only 5 atoms are used to keep the numbers small enough for exploration.

In [8]:
nesting_universe_instances = ['a', 'b', 'c', 'd', 'e']

Construct the full set of sequences possible with these atoms to be the Anything set.

In [9]:
nesting_universe_anything = [[]]
for i in range(1, 5):
    if i > 1:
        old_template = copy.deepcopy(nesting_universe_anything[i - 2])
        nesting_universe_anything.append([])
    for j, instance in enumerate(nesting_universe_instances):
        if i == 1:
            nesting_universe_anything[0].append(instance)
        else:
            for el in old_template:
                nesting_universe_anything[i - 1].append(instance + el)
            
len(nesting_universe_anything[3])

625

Select all sequences that go with the notional binary Association classifier (ending in 'a'):

In [10]:
assoc_seqs = []
for level in nesting_universe_anything:
    for seq in level:
        if seq[-1] == 'a':
            assoc_seqs.append(seq)
            
len(assoc_seqs)

156

Let the participant feature 1 have a Type that is a Classifier with minimal interpretation of {\[b\], \[c\]} and participant feature 2 have a Type that is a Classifier with minimal interpretation of {\[d\], \[e\]}.

In [11]:
BC_seqs = []
for level in nesting_universe_anything:
    for seq in level:
        if seq[-1] == 'b' or seq[-1] == 'c':
            BC_seqs.append(seq)
            
len(BC_seqs)

312

In [12]:
DE_seqs = []
for level in nesting_universe_anything:
    for seq in level:
        if seq[-1] == 'd' or seq[-1] == 'e':
            DE_seqs.append(seq)
            
len(DE_seqs)

312

Select all feature-length sequences that aren't classified by our Association but have it as a featuring type, meaning that 'a' is the left-most item in the sequence.

In [13]:
part1_seqs = []
for level in nesting_universe_anything:
    for seq in level:
        if len(seq) > 1 and (seq[-1] == 'b' or seq[-1] == 'c') and seq[0] == 'a':
            part1_seqs.append(seq)
            
len(part1_seqs)

62

In [14]:
part2_seqs = []
for level in nesting_universe_anything:
    for seq in level:
        if len(seq) > 1 and (seq[-1] == 'd' or seq[-1] == 'e') and seq[0] == 'a':
            part2_seqs.append(seq)
            
len(part2_seqs)

62

In [15]:
part1_min_seqs = []
for seq in part1_seqs:
    if (len(seq) == 2):
        part1_min_seqs.append(seq)

In [16]:
part2_min_seqs = []
for seq in part2_seqs:
    if (len(seq) == 2):
        part2_min_seqs.append(seq)

In [17]:
part2_min_seqs

['ad', 'ae']

#### Section 2.2.2.2 - Examples with Random BiLinks

In order to build up the story, we have an individual sequence from the set of (binary) Link sequences. This link then has one end pointing to a viable Feature for participant 1 and one for participant 2.

In [18]:
bilinks = []
for i in range(0, 100):
    rand_bilink = [assoc_seqs[random.randint(0, len(assoc_seqs) - 1)],
               part1_seqs[random.randint(0, len(part1_seqs) - 1)],
               part2_seqs[random.randint(0, len(part2_seqs) - 1)]]
    bilinks.append(rand_bilink)

Now we try the end values as listed in the spec. First, we randomize up some values for the two ends of the random link.

In [19]:
part1_bilinks = sorted(bilinks,key=lambda x: x[1])
part2_bilinks = sorted(bilinks,key=lambda x: x[2])

And then we try the navigation. The function below collects all members of the feature sequence that are linked to a given sequence in our collection of BiLinks. This should be the implementation of current end semantics where the ends point from one participant to the other.

The map below is where individuals of the BC classifier set point to the value list of their implied feature from the association end.

In [20]:
part1_sliced = {}
for uni in set([p[1] for p in part1_bilinks]):
    local = []
    for bilink in bilinks:
        if bilink[1] == uni:
            local.append(bilink[2])
    part1_sliced.update({uni: local})
    
part1_sliced

{'acac': ['abd'],
 'abbb': ['aeae', 'aabd', 'abee'],
 'abac': ['aabd'],
 'abab': ['aedd', 'abbd', 'acbd', 'abcd'],
 'abec': ['abe'],
 'acbb': ['aabd', 'aedd', 'aded'],
 'adac': ['aded'],
 'aeb': ['aeed', 'aded', 'aaae'],
 'aec': ['abed', 'ade'],
 'adbb': ['acd', 'accd'],
 'adb': ['abe'],
 'aaec': ['acad', 'adae', 'aade'],
 'ac': ['accd', 'abcd'],
 'adbc': ['aede', 'abde'],
 'aaab': ['adae'],
 'aedb': ['abed'],
 'aac': ['aedd'],
 'addc': ['abde', 'adde'],
 'aebc': ['adbe'],
 'acdc': ['abed', 'abce'],
 'adec': ['aadd'],
 'addb': ['aadd'],
 'abeb': ['abcd', 'adee'],
 'aeac': ['aeee', 'aced'],
 'aebb': ['ad', 'acdd', 'accd'],
 'acbc': ['acae'],
 'aabc': ['ad'],
 'accc': ['accd'],
 'abdb': ['aedd'],
 'aecb': ['aede', 'adee', 'abae'],
 'aceb': ['aced', 'aae', 'ade', 'aead', 'ace'],
 'abcc': ['adee', 'adbe', 'acce', 'abae', 'abae'],
 'abcb': ['aebe', 'aebe'],
 'acdb': ['aad', 'abe', 'aad'],
 'adab': ['abbd', 'adae'],
 'acab': ['aadd', 'acdd'],
 'aaac': ['ad'],
 'adcc': ['abd', 'adce', 'abbe',

The map below is where individuals of the DE classifier set point to the value list of their implied feature from the association end.

In [21]:
part2_sliced = {}
for uni in set([p[2] for p in part2_bilinks]):
    local = []
    for bilink in bilinks:
        if bilink[2] == uni:
            local.append(bilink[1])
    part2_sliced.update({uni: local})
    
part2_sliced

{'aebe': ['abcb', 'aadc', 'abcb'],
 'accd': ['ac', 'adbb', 'aebb', 'accc'],
 'adcd': ['aeab'],
 'abd': ['adcc', 'acac'],
 'adce': ['adcc', 'accb'],
 'aeed': ['aeb', 'aacb', 'adcc'],
 'aabd': ['acbb', 'abbb', 'abac', 'acc'],
 'abe': ['acc', 'abec', 'adb', 'acdb'],
 'aace': ['abc'],
 'aede': ['adbc', 'aecb'],
 'aeae': ['abbb'],
 'abad': ['ab'],
 'abed': ['aec', 'aedb', 'acdc'],
 'ade': ['aceb', 'aec'],
 'aded': ['aeb', 'acbb', 'adac'],
 'adae': ['aaec', 'adab', 'aaab'],
 'acce': ['abcc'],
 'acae': ['acbc'],
 'aead': ['aceb'],
 'adde': ['accb', 'ab', 'addc'],
 'adbe': ['abcc', 'aebc'],
 'acdd': ['accb', 'aacb', 'acab', 'aebb'],
 'aeee': ['aeac'],
 'acad': ['aaec'],
 'aadd': ['addb', 'acab', 'adec'],
 'acd': ['adbb'],
 'abcd': ['ac', 'aacb', 'abeb', 'abab'],
 'aced': ['aceb', 'aeac'],
 'acee': ['adcb'],
 'aad': ['acdb', 'acdb'],
 'abce': ['acdc'],
 'abee': ['abbb'],
 'aade': ['aacb', 'aaec', 'abb'],
 'aedd': ['abab', 'acbb', 'aac', 'abdb'],
 'abbd': ['adab', 'abab', 'abb', 'aadb'],
 'adee'

Here is the list of 100 randomly generated BiLinks. The first entry is instances of the BiLink, the second instances of the first participant feature, and the third is instances of the second participant feature.

In [22]:
sorted(bilinks,key=lambda x: x[0])

[['aaaa', 'adcc', 'abbe'],
 ['aaba', 'addb', 'aadd'],
 ['aaca', 'adbb', 'acd'],
 ['aaca', 'accb', 'ad'],
 ['aaca', 'adcc', 'aeed'],
 ['aada', 'aedb', 'abed'],
 ['abba', 'abb', 'abbd'],
 ['acaa', 'adcb', 'ad'],
 ['acda', 'aaec', 'aade'],
 ['adda', 'acab', 'aadd'],
 ['aeaa', 'acbb', 'aedd'],
 ['aeaa', 'adbc', 'abde'],
 ['aeba', 'accb', 'adde'],
 ['aeba', 'abcc', 'abae'],
 ['aeea', 'aeab', 'adcd'],
 ['aeea', 'accb', 'acdd'],
 ['aeea', 'adec', 'aadd'],
 ['ba', 'abab', 'abcd'],
 ['baaa', 'adab', 'abbd'],
 ['baba', 'abcc', 'abae'],
 ['baca', 'aceb', 'ace'],
 ['bada', 'acbb', 'aabd'],
 ['bada', 'aabc', 'ad'],
 ['bada', 'acab', 'acdd'],
 ['bbca', 'aecb', 'abae'],
 ['bbca', 'aebb', 'accd'],
 ['bbda', 'acc', 'abe'],
 ['bca', 'aecb', 'aede'],
 ['bcaa', 'abbb', 'abee'],
 ['bcba', 'abeb', 'adee'],
 ['bcca', 'aebb', 'acdd'],
 ['bcca', 'aceb', 'aead'],
 ['bcda', 'abcc', 'adee'],
 ['bcda', 'abec', 'abe'],
 ['bcea', 'aceb', 'aae'],
 ['bdca', 'aec', 'ade'],
 ['bdda', 'adbc', 'aede'],
 ['bdda', 'aaec', '

#### Section 4.2.3 Links

For general links, there can be more than two participants. The rules for mapping from the instances of the links and the participants to instances matched to features of end types is less clear. Here we can explore that a bit with an example inspired by *The Last Dance*, the NBA finals between the Seattle Supersonics and Chicago Bulls. Then, for fun, a follow-on game where a key matchup between Gary "The Glove" Payton and Michael Jordan is repeated when Jordan is on another team in another year.

To use the team, year, player example, here are some abbreviations:
b - Bulls
s - Sonics
p - Gary Payton
j - Michael Jordan
6 - 1996
2 - 2002
w - Wizards
a - Association

In [23]:
link_universe_instances = ['a', 'b', 's', 'p', 'j', '6', '2', 'w']

In [24]:
link_universe_anything = [[]]
link_universe_flat = []
for i in range(1, 5):
    if i > 1:
        old_template = copy.deepcopy(link_universe_anything[i - 2])
        link_universe_anything.append([])
    for j, instance in enumerate(link_universe_instances):
        if i == 1:
            link_universe_anything[0].append(instance)
            link_universe_flat.append(instance)
        else:
            for el in old_template:
                link_universe_anything[i - 1].append(instance + el)
                link_universe_flat.append(instance + el)

In [25]:
def is_team(instance):
    if instance[-1] == 'b' or instance[-1] == 's' or instance[-1] == 'w':
        return True
    return False

In [26]:
def is_player(instance):
    if instance[-1] == 'p' or instance[-1] == 'j':
        return True
    return False

In [27]:
def is_year(instance):
    if instance[-1] == '2' or instance[-1] == '6':
        return True
    return False

In [28]:
def is_link(instance):
    if instance[-1] == 'a':
        return True
    return False

In [29]:
teams = [instance for instance in link_universe_flat if is_team(instance)]
players = [instance for instance in link_universe_flat if is_player(instance)]
years = [instance for instance in link_universe_flat if is_year(instance)]
links = [instance for instance in link_universe_flat if is_link(instance)]

The nesting for the features above is interesting in how we can interpret it (depending on features provided in the larger user model). For example, 'b6j' could represent Michael Jordan as he was a player in 1996 for the Bulls.
We should note that there a number of sequences that may not make sense like 'bjj' (Jordan referencing himself while on the Bulls?) This might point to some useful default semantics for SysML part usage and the like.
Note that we have no restrictions yet, so there are several combinations where players appear to be linked to the wrong team. But we have said what the legal combinations are ... this may be oppositions rather than playing on the team.

In [30]:
players[40:50]

['bpp', 'bpj', 'bjp', 'bjj', 'b6p', 'b6j', 'b2p', 'b2j', 'bwp', 'bwj']

As before, the participants of our association restrict the universe of sequences to those that have the association minimum interpretation ('a') as the head and a minimal interpretation ('p', 'j' for players) as the tail.

In [31]:
linked_players = [linked_player for linked_player in players if linked_player[0] == 'a']
linked_teams = [linked_team for linked_team in teams if linked_team[0] == 'a']
linked_years = [linked_year for linked_year in years if linked_year[0] == 'a']

In [32]:
linked_years[10:20]

['aj6', 'aj2', 'a66', 'a62', 'a26', 'a22', 'aw6', 'aw2', 'aaa6', 'aaa2']

If we look at any one of these ends, they basically have a way to navigate to things of a given classifier either directly or through the other dimension. We can start by making a series of random links.

In [33]:
trilinks = []
for i in range(0, 100):
    rand_trilink = [links[random.randint(0, len(links) - 1)],
                    linked_players[random.randint(0, len(linked_players) - 1)],
                    linked_teams[random.randint(0, len(linked_teams) - 1)],
                    linked_years[random.randint(0, len(linked_years) - 1)]]
    trilinks.append(rand_trilink)

In [34]:
trilinks

[['aa6a', 'assp', 'awws', 'as26'],
 ['w26a', 'apbj', 'awab', 'a6p2'],
 ['jsa', 'a22j', 'ajbb', 'awj6'],
 ['p6aa', 'aaap', 'apjs', 'apw2'],
 ['6swa', 'asaj', 'a6w', 'aaj6'],
 ['sp6a', 'abbj', 'aswb', 'abj2'],
 ['pwwa', 'asaj', 'abpw', 'aa2'],
 ['aaa', 'a2pj', 'aaab', 'asa2'],
 ['pasa', 'aswp', 'aw6b', 'ajb2'],
 ['ajba', 'a6aj', 'a2bw', 'a6a2'],
 ['pbsa', 'aa2j', 'a2ww', 'ab62'],
 ['baja', 'ajwj', 'a6jb', 'aa6'],
 ['bpwa', 'absp', 'ajjb', 'apa6'],
 ['6waa', 'asj', 'a2pb', 'ajp2'],
 ['pasa', 'ajwj', 'awjw', 'a2a6'],
 ['wpwa', 'aj6p', 'ajww', 'as62'],
 ['6p6a', 'apaj', 'aasw', 'aap2'],
 ['b66a', 'abpp', 'a6jw', 'awb6'],
 ['b2ba', 'ajsp', 'apab', 'ajp2'],
 ['sbpa', 'a6wp', 'awpw', 'ap22'],
 ['pbba', 'ajjp', 'a22b', 'abp2'],
 ['b6aa', 'asjj', 'appw', 'a662'],
 ['sj6a', 'aj', 'abw', 'a2p6'],
 ['pa', 'apwj', 'a6s', 'asw6'],
 ['2j2a', 'a2sj', 'awsw', 'aa66'],
 ['wa6a', 'aa6j', 'ap2b', 'as22'],
 ['sp2a', 'aw2p', 'a2js', 'a2w6'],
 ['p2a', 'ab2j', 'absb', 'abw6'],
 ['sa2a', 'apjp', 'abas', 'awp2']

Next we consider how we made specify one of the ends for this link. It would make sense to have the Teams classifier be able to a Feature that calls out all players associated with the team, and perhaps to further filter this by year.

In [35]:
team_trilinks = sorted(trilinks,key=lambda x: x[2])
team_trilinks

[['pbba', 'ajjp', 'a22b', 'abp2'],
 ['622a', 'a62j', 'a26b', 'a226'],
 ['6p2a', 'apjj', 'a2bb', 'awp2'],
 ['ajba', 'a6aj', 'a2bw', 'a6a2'],
 ['jwpa', 'a6sj', 'a2bw', 'aw22'],
 ['sp2a', 'aw2p', 'a2js', 'a2w6'],
 ['swsa', 'ap6p', 'a2jw', 'aw62'],
 ['ba6a', 'aw6p', 'a2jw', 'abs6'],
 ['6waa', 'asj', 'a2pb', 'ajp2'],
 ['ajaa', 'awjj', 'a2sw', 'awb2'],
 ['2psa', 'ajap', 'a2w', 'a2j6'],
 ['bbaa', 'aa6j', 'a2wb', 'a2j6'],
 ['pbsa', 'aa2j', 'a2ww', 'ab62'],
 ['26wa', 'asbj', 'a66b', 'aj22'],
 ['bsba', 'ajj', 'a66s', 'aws6'],
 ['sa', 'aapj', 'a6as', 'as2'],
 ['bbwa', 'aj2p', 'a6bw', 'ajb2'],
 ['baja', 'ajwj', 'a6jb', 'aa6'],
 ['baa', 'abbj', 'a6jb', 'a26'],
 ['a6sa', 'asjj', 'a6js', 'aj22'],
 ['b66a', 'abpp', 'a6jw', 'awb6'],
 ['awsa', 'a2pj', 'a6jw', 'ab22'],
 ['pa', 'apwj', 'a6s', 'asw6'],
 ['6swa', 'asaj', 'a6w', 'aaj6'],
 ['aaa', 'a2pj', 'aaab', 'asa2'],
 ['awpa', 'awaj', 'aapb', 'apw2'],
 ['6p6a', 'apaj', 'aasw', 'aap2'],
 ['6paa', 'aswp', 'ab', 'aja6'],
 ['sa', 'assj', 'ab', 'a662'],
 ['22

In [38]:
team_sliced = {}
for uni in set([p[2] for p in team_trilinks]):
    local = []
    for trilink in trilinks:
        if trilink[2] == uni:
            local.append(trilink[1])
    team_sliced.update({uni: local})
    
team_sliced

{'a22b': ['ajjp'],
 'ajjs': ['awj'],
 'abbb': ['aw6p'],
 'a6w': ['asaj'],
 'ajww': ['aj6p'],
 'a6jb': ['ajwj', 'abbj'],
 'asab': ['awpp'],
 'a2bb': ['apjj'],
 'abpw': ['asaj', 'abbj'],
 'a6jw': ['abpp', 'a2pj'],
 'ajs': ['abp'],
 'aww': ['awaj'],
 'awpw': ['a6wp', 'abap'],
 'abps': ['ajjp'],
 'awab': ['apbj'],
 'abww': ['aaj'],
 'ap2w': ['asj'],
 'apsw': ['ajj'],
 'a6as': ['aapj'],
 'aswb': ['abbj'],
 'ab6s': ['absj'],
 'apwb': ['aaj'],
 'assb': ['aabj'],
 'abjw': ['abbp'],
 'awjb': ['apj'],
 'apjs': ['aaap'],
 'a6bw': ['aj2p'],
 'ajsb': ['ab2j'],
 'ajjw': ['a2pp'],
 'asas': ['apap'],
 'aj6b': ['absj'],
 'apab': ['ajsp'],
 'a2js': ['aw2p'],
 'awjw': ['ajwj'],
 'ajjb': ['absp'],
 'aw6b': ['aswp'],
 'aasw': ['apaj'],
 'a2pb': ['asj'],
 'aw2s': ['appj'],
 'awaw': ['ap'],
 'awjs': ['ajap'],
 'abw': ['aj', 'ajpj'],
 'a66b': ['asbj'],
 'asbs': ['as6j'],
 'awsb': ['aw6p'],
 'awps': ['abjp'],
 'as2s': ['aapj', 'a2pp'],
 'abs': ['aawp'],
 'a6js': ['asjj'],
 'ajbs': ['apjp'],
 'apps': ['awpp'],
